In [1]:
import pyrosetta 
import rosetta 
from sys import argv
import os 
os.environ[ "SLURM_ARRAY_TASK_ID" ] = "96" # for testing! 

In [2]:
with open( '../../mutant_list.txt' ) as fn:
    mutant_list = fn.read().split()
    mutant_name = mutant_list[ int( os.environ[ 'SLURM_ARRAY_TASK_ID' ] ) - 1 ] 

In [3]:
fmt = dict( zip( 'ANDRCQEGHILKMPFSTWYV', [
    'ALA','ASN','ASP','ARG','CYS','GLN','GLU',
    'GLY','HIS','ILE','LEU','LYS','MET','PRO','PHE','SER',
    'THR','TRP','TYR','VAL' ] ) ) 

In [4]:
# flags from command line 
with open( 'input_files/flags' ) as fn:
    flags = fn.read().replace( '\n', ' ' )
pyrosetta.init( ''.join( flags ) ) 

# extra residue (pNPG) params 
ligand_params = pyrosetta.Vector1( [ 'input_files/pNPG.params' ] )
new_res_set = pyrosetta.generate_nonstandard_residue_set( ligand_params )

# construct pose 
p = pyrosetta.Pose()
pyrosetta.pose_from_file( p, new_res_set, 'input_files/bglb.pdb' ) 

# construct score function 
scorefxn = pyrosetta.create_score_function( 'beta_cst' ) 

# add in enzyme design constraints 
add_cst = rosetta.protocols.enzdes.AddOrRemoveMatchCsts()
add_cst.cstfile( 'input_files/pNPG.enzdes.cst' ) 
add_cst.set_cst_action( rosetta.protocols.enzdes.CstAction.ADD_NEW )
add_cst.apply( p ) 

# mutate the residue 
target = int( mutant_name[ 1:-1 ] )
new_res = fmt[ mutant_name[ -1 ] ] 
mut = rosetta.protocols.simple_moves.MutateResidue( target, new_res )
mut.apply( p ) 

PyRosetta-4 2016 [Rosetta 2016 unknown:79465c30147c9c82133981175c9eefae20b0928a 2016-12-08 11:57:48 -0800] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [5]:
# set up pack task 
tf = rosetta.core.pack.task.TaskFactory()
around = rosetta.protocols.toolbox.task_operations.DesignAroundOperation()
around.include_residue( 446 ) #ligand 
around.include_residue( target ) 
around.repack_shell( 15 ) # let's grid search this  
around.resnums_allow_design( False )
around.allow_design( False )
tf.push_back( around ) 
pt = tf.create_task_and_apply_taskoperations(p)
print( pt )

# repack and minimize 
repack = rosetta.protocols.enzdes.EnzRepackMinimize()
repack.set_scorefxn_repack( scorefxn )
repack.set_scorefxn_minimize( scorefxn )
repack.set_min_bb( True )
repack.set_min_lig( True )  
repack.set_min_rb( True ) 
repack.set_min_sc( True )
repack.task_factory( tf ) # adds packer task 

# monte carlo 
parsed = rosetta.protocols.simple_moves.GenericMonteCarloMover()
parsed.set_mover( repack )
parsed.set_maxtrials( 10 )
parsed.set_scorefxn( scorefxn )
parsed.apply( p ) 

# output PDB, probably for local testing only 
#p.dump_pdb( '/Users/alex/Desktop/output_files/{}.pdb'.format( mutant_name) ) 

#Packer_Task

resid	pack?	design?	allowed_aas
1	FALSE	FALSE	
2	FALSE	FALSE	
3	FALSE	FALSE	
4	FALSE	FALSE	
5	FALSE	FALSE	
6	FALSE	FALSE	
7	FALSE	FALSE	
8	FALSE	FALSE	
9	FALSE	FALSE	
10	FALSE	FALSE	
11	FALSE	FALSE	
12	FALSE	FALSE	
13	TRUE	FALSE	THR
14	TRUE	FALSE	SER
15	TRUE	FALSE	THR
16	TRUE	FALSE	SER
17	TRUE	FALSE	SER
18	TRUE	FALSE	TYR
19	TRUE	FALSE	GLN
20	TRUE	FALSE	ILE
21	FALSE	FALSE	
22	FALSE	FALSE	
23	FALSE	FALSE	
24	FALSE	FALSE	
25	FALSE	FALSE	
26	FALSE	FALSE	
27	FALSE	FALSE	
28	FALSE	FALSE	
29	FALSE	FALSE	
30	FALSE	FALSE	
31	FALSE	FALSE	
32	FALSE	FALSE	
33	FALSE	FALSE	
34	TRUE	FALSE	TRP
35	FALSE	FALSE	
36	FALSE	FALSE	
37	TRUE	FALSE	PHE
38	FALSE	FALSE	
39	FALSE	FALSE	
40	FALSE	FALSE	
41	FALSE	FALSE	
42	FALSE	FALSE	
43	FALSE	FALSE	
44	TRUE	FALSE	VAL
45	FALSE	FALSE	
46	FALSE	FALSE	
47	FALSE	FALSE	
48	FALSE	FALSE	
49	FALSE	FALSE	
50	TRUE	FALSE	GLY
51	FALSE	FALSE	
52	FALSE	FALSE	
53	FALSE	FALSE	
54	FALSE	FALSE	
55	FALSE	FALSE	
56	FALSE	FALSE	
57	FALSE	FALSE	
58	FALSE	FALSE	
59	FALSE	FA

In [56]:
build_list = []
for i in range( 1, 299 ):
    my_tuple = ( 
        str( rosetta.core.scoring.ScoreType(i) ).split( '.' )[ 1 ], 
        rosetta.core.scoring.ScoreFunction.score_by_scoretype(scorefxn,p,rosetta.core.scoring.ScoreType(i)),         
    )
    if my_tuple[1] != 0.0:
        build_list.append( my_tuple )

In [57]:
build_list

[('fa_atr', -2880.5141242894924),
 ('fa_rep', 386.30358863935066),
 ('fa_sol', 2060.591324461968),
 ('fa_intra_atr_xover4', -170.80056726731385),
 ('fa_intra_rep_xover4', 89.59680847674382),
 ('fa_intra_sol_xover4', 117.82649811845987),
 ('lk_ball', 1095.5396821038419),
 ('lk_ball_iso', -1125.1061336019927),
 ('lk_ball_bridge', -13.58448036587842),
 ('lk_ball_bridge_uncpl', -45.08308604606859),
 ('fa_elec', -830.7913895522489),
 ('fa_intra_elec', -104.72821662165035),
 ('pro_close', 101.25281695018423),
 ('hbond_sr_bb', -188.63719742885556),
 ('hbond_lr_bb', -102.68562971911676),
 ('hbond_bb_sc', -61.549985495720115),
 ('hbond_sc', -92.08718835221174),
 ('dslf_fa13', -1.245661166410178),
 ('atom_pair_constraint', 0.012921344717011715),
 ('dihedral_constraint', 0.015253478219377267),
 ('omega', 40.59339643807278),
 ('fa_dun_dev', 62.37722535949133),
 ('fa_dun_rot', 185.18529019877258),
 ('fa_dun_semi', 454.88760281527624),
 ('p_aa_pp', -86.60990047396345),
 ('hxl_tors', 84.8911440732766

In [65]:
energies = p.energies()
for i in range( 1, p.total_residue() ):
    for j in range( 200 ):
        residue_energies = p.energies().residue_total_energies( i )
        score_type = rosetta.core.scoring.ScoreType( j )
        score_type_str = str( score_type ).split( '.' )[ 1 ]
        score_value = residue_energies[ score_type ] 
        if i > 0:
            if score_value != 0.0:
                build_list.append( ( i, score_type_str, score_value ) )
            
build_list

[('fa_atr', -2880.5141242894924),
 ('fa_rep', 386.30358863935066),
 ('fa_sol', 2060.591324461968),
 ('fa_intra_atr_xover4', -170.80056726731385),
 ('fa_intra_rep_xover4', 89.59680847674382),
 ('fa_intra_sol_xover4', 117.82649811845987),
 ('lk_ball', 1095.5396821038419),
 ('lk_ball_iso', -1125.1061336019927),
 ('lk_ball_bridge', -13.58448036587842),
 ('lk_ball_bridge_uncpl', -45.08308604606859),
 ('fa_elec', -830.7913895522489),
 ('fa_intra_elec', -104.72821662165035),
 ('pro_close', 101.25281695018423),
 ('hbond_sr_bb', -188.63719742885556),
 ('hbond_lr_bb', -102.68562971911676),
 ('hbond_bb_sc', -61.549985495720115),
 ('hbond_sc', -92.08718835221174),
 ('dslf_fa13', -1.245661166410178),
 ('atom_pair_constraint', 0.012921344717011715),
 ('dihedral_constraint', 0.015253478219377267),
 ('omega', 40.59339643807278),
 ('fa_dun_dev', 62.37722535949133),
 ('fa_dun_rot', 185.18529019877258),
 ('fa_dun_semi', 454.88760281527624),
 ('p_aa_pp', -86.60990047396345),
 ('hxl_tors', 84.8911440732766